In [1]:
import numpy as np
import pandas as pd

inputdata = pd.read_csv("MDinfo.csv").query('day==2 & tms!=3') # get study info, for day 1 & groups 1 & 2
images = np.array(inputdata['image']) # get array of .nii files from image column
group = np.array(inputdata['tms']) # get array of subject group indices (1=M1,2=SPL)

print(images)
print(images.size)
print(group)
print(group.size)

['INPUT/KURTOSIS_DKITOOLSFULL_MD/3_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/4_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/5_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/8_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/9_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/10_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/11_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/12_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/13_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/14_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/15_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/16_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'INPUT/KURTOSIS_DKITOOLSFULL_MD/17_KURTOSIS_DKITOOLSFULL_MD_day2change.nii'
 'IN

In [2]:
# Split data into training set and test set
from sklearn.utils import check_random_state
from sklearn.cross_validation import train_test_split

rng = check_random_state(42)
images_train, images_test, group_train, group_test = train_test_split(images, group, train_size=.7, random_state=rng)

# Sort test data for better visualization
perm = np.argsort(group_test)
group_test = group_test[perm]
images_test = images_test[perm]

/Users/Nikola/anaconda/envs/mne/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# Fit the SpaceNet and predict with it
from nilearn.decoding import SpaceNetClassifier

# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory="nilearn_cache", penalty='tv-l1', memory_level=2)
decoder.fit(images_train, group_train)  # fit
y_pred = decoder.predict(images_test) # predict
accuracy = (y_pred == group_test).mean() * 100.
print("tv-l1 classification accuracy : %g%%" % accuracy)

[NiftiMasker.fit] Loading data from [INPUT/KURTOSIS_DKITOOLSFULL_MD/29_KURTOSIS_DKITOOLSFULL_MD_day2change.nii, INPUT/KURTOSIS_DKITOOLSFULL_MD/23_KURTOSIS_DKITOOLSFULL_MD_day2change.nii, INPUT/KURTOSIS_DKITOOLSFULL_MD/19_KURTOSIS_DKITOO
[NiftiMasker.fit] Computing the mask
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(79, 95, 79, 30),
affine=array([[  -2.,    0.,    0.,   78.],
       [   0.,    2.,    0., -112.],
       [   0.,    0.,    2.,  -70.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals


/Users/Nikola/.local/lib/python3.5/site-packages/nilearn/decoding/space_net.py:796: UserWarning: Brain mask is bigger than the volume of a standard human brain. This object is probably not tuned to be used on such data.
  self.screening_percentile, self.mask_img_, verbose=self.verbose)


________________________________________________________________________________
[Memory] Calling nilearn.decoding.space_net.path_scores...
path_scores(functools.partial(<function tvl1_solver at 0x11827b268>, loss='logistic'), array([[ 0.185695, ..., -0.185695],
       ..., 
       [ 0.185695, ..., -0.185695]]), 
array([ 1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1]), 
array([[[False, ..., False],
        ..., 
        [False, ..., False]],

       ..., 
       [[False, ..., False],
        ..., 
        [False, ..., False]]], dtype=bool), 
None, [0.5], array([ 2,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29]), 
array([0, 1, 3, 8]), {'max_iter': 1000, 'tol': 0.0001}, is_classif=True, debias=False, eps=0.001, screening_percentile=11.41809931113089, n_alphas=10, key=(0, 0), verbose=1)


/Users/Nikola/anaconda/envs/mne/lib/python3.5/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function center_data is deprecated; center_data was deprecated in version 0.18 and will be removed in 0.20. Use utilities in preprocessing.data instead
  warnings.warn(msg, category=DeprecationWarning)
..........

__________________________________________________path_scores - 1415.7s, 23.6min


...

/Users/Nikola/anaconda/envs/mne/lib/python3.5/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function center_data is deprecated; center_data was deprecated in version 0.18 and will be removed in 0.20. Use utilities in preprocessing.data instead
  warnings.warn(msg, category=DeprecationWarning)
..........

____________________________________________________path_scores - 215.1s, 3.6min
________________________________________________________________________________
[Memory] Calling nilearn.decoding.space_net.path_scores...
path_scores(functools.partial(<function tvl1_solver at 0x11827b268>, loss='logistic'), array([[ 0.185695, ..., -0.185695],
       ..., 
       [ 0.185695, ..., -0.185695]]), 
array([ 1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1]), 
array([[[False, ..., False],
        ..., 
        [False, ..., False]],

       ..., 
       [[False, ..., False],
        ..., 
        [False, ..., False]]], dtype=bool), 
None, [0.5], array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 23, 24, 26, 27, 28, 29]), 
array([19, 21, 22, 25]), {'max_iter': 1000, 'tol': 0.0001}, is_classif=True, debias=False, eps=0.001, screening_percentile=11.41809931113089, n_alphas=10, key

/Users/Nikola/anaconda/envs/mne/lib/python3.5/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function center_data is deprecated; center_data was deprecated in version 0.18 and will be removed in 0.20. Use utilities in preprocessing.data instead
  warnings.warn(msg, category=DeprecationWarning)
..........

____________________________________________________path_scores - 354.3s, 5.9min
________________________________________________________________________________
[Memory] Calling nilearn.decoding.space_net.path_scores...
path_scores(functools.partial(<function tvl1_solver at 0x11827b268>, loss='logistic'), array([[ 0.185695, ..., -0.185695],
       ..., 
       [ 0.185695, ..., -0.185695]]), 
array([ 1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1]), 
array([[[False, ..., False],
        ..., 
        [False, ..., False]],

       ..., 
       [[False, ..., False],
        ..., 
        [False, ..., False]]], dtype=bool), 
None, [0.5], array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 25, 28, 29]), 
array([23, 24, 26, 27]), {'max_iter': 1000, 'tol': 0.0001}, is_classif=True, debias=False, eps=0.001, screening_percentile=11.41809931113089, n_alphas=10, key

/Users/Nikola/anaconda/envs/mne/lib/python3.5/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function center_data is deprecated; center_data was deprecated in version 0.18 and will be removed in 0.20. Use utilities in preprocessing.data instead
  warnings.warn(msg, category=DeprecationWarning)
..........

In [ ]:
# Visualization of Graph-net weights
from nilearn import plotting
from nilearn.plotting import plot_stat_map, show

coef_img = decoder.coef_img_
plot_stat_map(coef_img, title="graph-net: accuracy %g%%" % accuracy, cut_coords=(-52, -5), display_mode="yz")

plotting.plot_glass_brain(
    coef_img, title='plot_glass_brain',
    black_bg=True, threshold=3)

# Save the coefficients to a nifti file
coef_img.to_filename('MD_testday2_tv-11_weights.nii')